# **csv Data Load**

- 개인 parsed dataset csv 파일 경로에 맞추어 불러와주세요
- df 변수 저장 후 아래 각 케이스마다 코드 셀을 그대로 복붙하면 해당 데이터셋을 사용할 수 있습니다

In [1]:
# 개인 디렉토리에 맞추어 경로를 설정
from google.colab import drive
drive.mount('/content/gdrive/')

import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/인공지능 커리어패스/parsed_enron_all.csv')

Mounted at /content/gdrive/


# **[1] 전체 Enron 데이터셋**
- 변수명: enron_dataset
- To 항목 공란(NaN) 삭제
- 자기자신에게 보낸 메일 삭제
- Date 정보 utc 기준으로 통일

In [2]:
def clean_enron_emails(df):
    df_processed = df.copy()
    initial_count = len(df_processed)
    print(f"초기 데이터 개수: {initial_count}")
    print("-" * 50)

    # 1. 'To' 항목이 공란(NaN)인 메일 개수 계산 (삭제 대상)
    to_nan_mask = df_processed['To'].isna()
    to_nan_count = to_nan_mask.sum()
    print(f"To 항목이 공란(NaN)인 메일")
    print(f"삭제 대상 메일 개수: {to_nan_count}")
    print("-" * 50)

    # 2 & 3. process_recipients 함수를 적용하여 'To' 필드 처리
    def process_recipients(row):
        from_email = str(row['From']).strip().lower()
        to_field = row['To']

        # To 필드가 문자열이 아닌 경우(NaN 등)는 변경하지 않음
        if not isinstance(to_field, str) or not to_field:
            return to_field

        recipients = {email.strip().lower() for email in to_field.split(',')}

        if from_email in recipients:
            if len(recipients) == 1:
                return None  # 자기 자신에게만 보낸 경우 -> 삭제 대상
            else:
                recipients.remove(from_email)
                return ', '.join(recipients) # 자기 자신과 타인에게 보낸 경우 -> 수정
        else:
            return to_field # 그 외 -> 유지
    df_processed['To_processed'] = df_processed.apply(process_recipients, axis=1)

    # 2. 자기 자신에게만 보낸 메일 개수 계산 (삭제 대상)
    # To_processed가 None이면서, 원래 To는 NaN이 아니었던 경우
    self_only_mask = df_processed['To_processed'].isna() & df_processed['To'].notna()
    self_only_count = self_only_mask.sum()
    print(f"자기 자신에게만 보낸 메일")
    print(f"삭제 대상 메일 개수: {self_only_count}")
    print("-" * 50)

    # 3. 자기 자신과 다른 사람에게 보낸 메일 개수 계산 (수정 대상)
    modified_mask = (df_processed['To_processed'].notna()) & (df_processed['To_processed'] != df_processed['To'])
    modified_count = modified_mask.sum()
    print(f"자기 자신과 동시에 타인에게 보낸 메일의 수신자 목록에서 자기 자신을 제외")
    print(f"수정된 메일 개수: {modified_count}")
    print("-" * 50)

    # 4. 최종 데이터셋 생성
    # 삭제 대상(To가 NaN이거나, 자기 자신에게만 보낸 메일)을 모두 제외
    final_df = df_processed[~to_nan_mask & ~self_only_mask].copy()

    # 처리된 'To_processed' 컬럼을 'To' 컬럼에 반영
    final_df['To'] = final_df['To_processed']
    # 원본 컬럼 목록을 유지하기 위해 불필요한 임시 컬럼 삭제
    final_df = final_df[df.columns]

    final_count = len(final_df)
    total_removed = initial_count - final_count

    print(f"최종 데이터 개수: {final_count}")
    print(f"총 {total_removed}개의 메일이 삭제되었습니다.")

    return final_df


enron_dataset = clean_enron_emails(df)

# Date 정보를 utc 기준으로 통일
enron_dataset['Date'] = pd.to_datetime(
    enron_dataset['Date'],
    format='%a, %d %b %Y %H:%M:%S %z',
    errors='coerce',
    utc=True
)

초기 데이터 개수: 517396
--------------------------------------------------
To 항목이 공란(NaN)인 메일
삭제 대상 메일 개수: 21847
--------------------------------------------------
자기 자신에게만 보낸 메일
삭제 대상 메일 개수: 16295
--------------------------------------------------
자기 자신과 동시에 타인에게 보낸 메일의 수신자 목록에서 자기 자신을 제외
수정된 메일 개수: 12160
--------------------------------------------------
최종 데이터 개수: 479254
총 38142개의 메일이 삭제되었습니다.


In [3]:
# [1] enron_dataset의 구성을 확인
enron_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479254 entries, 0 to 517395
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   Message-ID  479254 non-null  object             
 1   Date        479254 non-null  datetime64[ns, UTC]
 2   From        479254 non-null  object             
 3   To          479254 non-null  object             
 4   Subject     460708 non-null  object             
 5   X-From      479247 non-null  object             
 6   X-To        479247 non-null  object             
 7   X-cc        117535 non-null  object             
 8   X-bcc       168 non-null     object             
 9   X-Folder    479247 non-null  object             
 10  X-Origin    479247 non-null  object             
 11  X-FileName  476689 non-null  object             
 12  Body        479254 non-null  object             
dtypes: datetime64[ns, UTC](1), object(12)
memory usage: 51.2+ MB


# **메일데이터가 가장 많은 상위 5명 분석**

- From항목 + To항목 합쳐서 주고받은 메일이 가장 많은 Top5 ranking 목적
- 주의: From항목 또는 To항목 한 항목에만 너무 자주 등장하는 경우가 있을 수 있으므로 From, To, From+To 나누어서 분석 진행

In [14]:
from itertools import chain

In [13]:
# From 항목에 가장 많이 등장한 Top5 메일주소
from_counts = enron_dataset['From'].str.strip().str.lower().value_counts()
top5_from = from_counts.head(5)
print('From 항목에 가장 많이 등장한 Top5 메일주소 목록')
print(top5_from)

From 항목에 가장 많이 등장한 Top5 메일주소 목록
From
kay.mann@enron.com           16661
vince.kaminski@enron.com     14223
jeff.dasovich@enron.com      11285
sara.shackleton@enron.com     8741
chris.germany@enron.com       8693
Name: count, dtype: int64


In [15]:
# To 항목에 가장 많이 등장한 Top5 메일주소
to_non_null = enron_dataset['To'].dropna().str.split(',')
to_flat = pd.Series(chain.from_iterable(to_non_null)).str.strip().str.lower()
to_counts = to_flat.value_counts()
top5_to = to_counts.head(5)
print('To 항목에 가장 많이 등장한 Top5 메일주소 목록')
print(top5_to)

To 항목에 가장 많이 등장한 Top5 메일주소 목록
richard.shapiro@enron.com    14686
jeff.dasovich@enron.com      14005
tana.jones@enron.com         12807
steven.kean@enron.com        12657
sara.shackleton@enron.com    11417
Name: count, dtype: int64


In [16]:
# From항목 + To항목 주고받은 메일이 가장 많은 Top5 메일주소
combined_counts = from_counts.add(to_counts, fill_value=0)
top5_overall = combined_counts.sort_values(ascending=False).head(5).astype(int)

for i, email in enumerate(top5_overall.index, 1):
    total = int(top5_overall[email])
    from_count = int(from_counts.get(email, 0))
    to_count = int(to_counts.get(email, 0))

    from_ratio = from_count / total * 100 if total else 0
    to_ratio = to_count / total * 100 if total else 0

    print(email)
    print(f"총 메일 수: {total}")
    print(f"발신메일 수: {from_count}({from_ratio:.1f}%)")
    print(f"수신메일 수: {to_count}({to_ratio:.1f}%)\n")

jeff.dasovich@enron.com
총 메일 수: 25290
발신메일 수: 11285(44.6%)
수신메일 수: 14005(55.4%)

kay.mann@enron.com
총 메일 수: 23087
발신메일 수: 16661(72.2%)
수신메일 수: 6426(27.8%)

tana.jones@enron.com
총 메일 수: 21243
발신메일 수: 8436(39.7%)
수신메일 수: 12807(60.3%)

sara.shackleton@enron.com
총 메일 수: 20158
발신메일 수: 8741(43.4%)
수신메일 수: 11417(56.6%)

steven.kean@enron.com
총 메일 수: 18833
발신메일 수: 6176(32.8%)
수신메일 수: 12657(67.2%)

